In [31]:
import pandas as pd
import numpy as np
import requests
import json
import os
import io

In [117]:
''' CONEXION A LA API '''
try: #Probar si la API funciona correctamente
    package_information_url = 'http://datos.gob.ar/api/3/action/package_show?id=cultura-mapa-cultural-espacios-culturales'
    package_information = requests.get(package_information_url)
    package_dict = json.loads(package_information.content)
    package_dict = package_dict['result']
except:
    print("Error, no se pudo conectar a la API")

dataset = []
for dic in package_dict['resources']:
    interest_list =  ['Museo', 'Salas de Cine', 'Bibliotecas Populares']
    if dic['name'] in interest_list:
        print(dic['url'])
        dataset.append(dic['url'])

biblioteca_popular = dataset[0]
museos = dataset[1]
cine = dataset[2]

https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/biblioteca_popular.csv
https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museo.csv
https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/392ce1a8-ef11-4776-b280-6f1c7fae16ae/download/cine.csv


### Procesamiento de datos

In [137]:

urlBiblio = requests.get(biblioteca_popular).content
df_biblio = pd.read_csv(io.StringIO(urlBiblio.decode('utf-8')))

urlMuseo = requests.get(museos).content
df_museo = pd.read_csv(io.StringIO(urlMuseo.decode('utf-8')))

urlCine = requests.get(cine).content
df_cine = pd.read_csv(io.StringIO(urlCine.decode('utf-8')))


In [131]:
#Creare un dataframe vacio con las columnas a integrar para la normalizacion solo a modo de ayuda
df_normalize = pd.DataFrame(columns=["cod_localidad","id_provincia","id_departamento",
                                     "categoría","provincia","localidad",
                                     "nombre","domicilio","código postal",
                                     "número de teléfono","mail","web"])

#### Normalizar las columnas para luego unirlas

In [17]:
df_normalize.columns

Index(['cod_localidad', 'id_provincia', 'id_departamento', 'categoría',
       'provincia', 'localidad', 'nombre', 'domicilio', 'código postal',
       'número de teléfono', 'mail', 'web'],
      dtype='object')

In [19]:
#Renombrar las columnas para la normalizacion
# df.rename(columns={'col1':'cod_localidad',
#                    'col2':'id_provincia',
#                    'col3':'id_departamento',
#                    'col4':'categoría',
#                    'col5':'provincia',
#                    'col6':'localidad',
#                    'col7':'nombre',
#                    'col8':'domicilio',
#                    'col9':'código postal',
#                    'col10':'número de teléfono',
#                    'col11':'mail',
#                    'col12':'web'})

In [145]:
df_biblio = df_biblio.rename(columns={'Cod_Loc':'cod_localidad',
                   'IdProvincia':'id_provincia',
                   'IdDepartamento':'id_departamento',
                   'Categoría':'categoría',
                   'Provincia':'provincia',
                   'Localidad':'localidad',
                   'Nombre':'nombre',
                   'Domicilio':'domicilio',
                   'CP':'código_postal',
                   'Teléfono':'número_de_teléfono',
                   'Mail':'mail',                                      
                   'Web':'web',
                    'Fuente':'fuente'})

In [146]:
df_museo = df_museo.rename(columns={'cod_loc':'cod_localidad',
                   'idprovincia':'id_provincia',
                   'iddepartamento':'id_departamento',
                   'categoria':'categoría',
                   'direccion':'domicilio',
                   'CP':'código_postal',
                   'telefono':'número_de_teléfono'})

In [147]:
df_cine = df_cine.rename(columns={'Cod_Loc':'cod_localidad',
                   'IdProvincia':'id_provincia',
                   'IdDepartamento':'id_departamento',
                   'Categoría':'categoría',
                   'Provincia':'provincia',
                   'Localidad':'localidad',
                   'Nombre':'nombre',
                   'Dirección':'domicilio',
                   'CP':'código_postal',
                   'Teléfono':'número_de_teléfono',
                   'Mail':'mail',
                   'Web':'web',
                   'Fuente':'fuente',
                   'Pantallas':'pantallas',
                   'Butacas':'butacas',
                   'espacio_INCAA':'espacio_incaa'})

### Datos Conjuntos

###### Uno las tablas en su conjunto

In [148]:
frames = [df_cine, df_biblio, df_museo]
result = pd.concat(frames)

In [149]:
result.head(3)

,cod_localidad,id_provincia,id_departamento,Observaciones,categoría,provincia,Departamento,localidad,nombre,domicilio,...,Año_actualizacion,subcategoria,piso,latitud,longitud,tipolatitudlongitud,Info_adicional,jurisdiccion,ano_inauguracion,IDSInCA
0,10049030,10,10049,NaN,Salas de cine,Catamarca,Capital,Catamarca,Cinemacenter,Intendente Mamerto Medina 220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10091150,10,10091,NaN,Salas de cine,Catamarca,Santa Maria,Santa María,Centro Cultural San Agustín,San Martin 173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22140060,22,22140,NaN,Salas de cine,Chaco,San Fernando,Resistencia,Los Cines De La Costa,Av. Sarmiento 2600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Tabla normalizada con informacion de Museos, Salas de cine y Bibliotecas

In [151]:
df_normalizado = result[['cod_localidad',
 'id_provincia',
 'id_departamento',
 'categoría',
 'provincia',
 'localidad',
 'nombre',
 'domicilio',
 'código_postal',
 'número_de_teléfono',
 'mail',
 'web']]

In [152]:
df_normalizado.head(1)

,cod_localidad,id_provincia,id_departamento,categoría,provincia,localidad,nombre,domicilio,código_postal,número_de_teléfono,mail,web
0,10049030,10,10049,Salas de cine,Catamarca,Catamarca,Cinemacenter,Intendente Mamerto Medina 220,4700,s/d,s/d,http://www.cinemacenter.com.ar/


### Procesar datos conjuntos 

###### Esta es quizas la forma que mas se realice para la transformacion y obtencion de los registros totales, pero yo la veo innecesaria, pudiendo trabajar directamente sobre la base de datos para obtener la misma informacion, de todas formas la dejo abajo:

In [154]:
categoria = pd.DataFrame(result.groupby("categoría").agg("cod_localidad").size())
df_categoria = categoria.rename(columns={'cod_localidad':'registros_totales'}).reset_index()
df_categoria

,categoría,registros_totales
0,Bibliotecas Populares,2017
1,Espacios de Exhibición Patrimonial,1182
2,Salas de cine,329


In [155]:
fuente = pd.DataFrame(result.groupby("fuente").agg("cod_localidad").size())
df_fuente = fuente.rename(columns={'cod_localidad':'registros_totales'}).reset_index()
df_fuente

,fuente,registros_totales
0,CNMLH - Enlace SInCA,1
1,CNMLH - Ente Cultural de Tucumán,1
2,CONABIP,2010
3,DNPyM,1048
4,Dirección Provincial de Patrimonio Cultural - ...,1
5,Dirección de Cultura de Villa Allende,2
6,Dirección de Cultura y de Educación de la Muni...,1
7,Gob. Pcia.,1
8,Gobierno de la Provincia,14
9,Gobierno de la Provincia de Chubut / SInCA 2013,3


In [156]:
provinciaCategoria = pd.DataFrame(result.groupby(["provincia",'categoría']).agg('cod_localidad').count())
df_provinciaCategoria = provinciaCategoria.rename(columns={'cod_localidad':'registros_totales'}).reset_index()
df_provinciaCategoria

,provincia,categoría,registros_totales
0,Buenos Aires,Bibliotecas Populares,543
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,106
3,Catamarca,Bibliotecas Populares,38
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
68,Tierra del Fuego,Salas de cine,3
69,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,14
70,Tucumán,Bibliotecas Populares,37
71,Tucumán,Espacios de Exhibición Patrimonial,26


### Procesar informacion de cines

In [160]:
df_cine_procesado = df_cine[['provincia','pantallas','butacas','espacio_incaa']].copy()
df_cine_procesado['espacio_incaa'].replace({'0': 0, 'si' : 1, "SI": 1}, inplace=True)
df_cine_procesado


,provincia,pantallas,butacas,espacio_incaa
0,Catamarca,5,743,NaN
1,Catamarca,1,440,0.0
2,Chaco,5,820,NaN
3,Chubut,1,80,NaN
4,Corrientes,1,240,NaN
...,...,...,...,...
324,Buenos Aires,1,0,NaN
325,Buenos Aires,8,2037,NaN
326,Buenos Aires,1,430,1.0
327,Catamarca,1,1103,1.0


###### Creo que esta es la forma correcta de transformar estos datos, ya que de aqui se podra realizar las consultas sql obteniendo asi la informacion de:
- Cantidad de registros totales por categoría
- Cantidad de registros totales por fuente
- Cantidad de registros por provincia y categoría


In [153]:
registros = pd.DataFrame(result.groupby(['provincia','categoría','fuente']).agg('cod_localidad').size()).reset_index()
df_registros = registros.rename(columns={'cod_localidad':'registros_totales'})
df_registros

,provincia,categoría,fuente,registros_totales
0,Buenos Aires,Bibliotecas Populares,CONABIP,543
1,Buenos Aires,Espacios de Exhibición Patrimonial,DNPyM,146
2,Buenos Aires,Espacios de Exhibición Patrimonial,Red de Espacios Culturales Vicente López,4
3,Buenos Aires,Espacios de Exhibición Patrimonial,Secretaria de Cultura del Municipio de Esteban...,1
4,Buenos Aires,Espacios de Exhibición Patrimonial,UNLA,1
...,...,...,...,...
99,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,DNPyM,14
100,Tucumán,Bibliotecas Populares,CONABIP,37
101,Tucumán,Espacios de Exhibición Patrimonial,CNMLH - Ente Cultural de Tucumán,1
102,Tucumán,Espacios de Exhibición Patrimonial,DNPyM,25
